In [1]:
# Si ya tienes requests y bs4 instalados, esta celda no hará nada.
# Si no, instalará automáticamente.
import sys, subprocess

try:
    import requests, bs4
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "requests", "beautifulsoup4"])
    import requests, bs4

import json, time, pathlib
from bs4 import BeautifulSoup

In [2]:
# — CONFIGURA:
COMP_ID = "1410"    # ID de tu liga
PA_LIGA = 75        # media de puntos encajados/juego
DELAY   = 0.5       # segundos entre peticiones

BASE = "https://www.fecanbaloncesto.com"
API  = BASE + "/services/estadisticas_partido.php?idPartido="
COMP = f"{BASE}/competicion/?id={COMP_ID}"

def partido_ids():
    html = requests.get(COMP, timeout=10).text
    soup = BeautifulSoup(html, "html.parser")
    return [a["data-idpartido"] for a in soup.select("a.estadisticas")]

def calc_eff(p):
    FGM = p["t2conv"] + p["t3conv"]
    FGA = p["t2int"]  + p["t3int"]
    return (p["puntos"]
            - (FGA-FGM)
            - (p["tlint"]-p["tlconv"])
            - p["faltas"]
            + 0.1 * p["min"])

out = []
for pid in partido_ids():
    data = requests.get(API+pid, timeout=10).json()
    for side in ("local","visitante"):
        pa_rival = data["marcadorVisitante"] if side=="local" else data["marcadorLocal"]
        FD = PA_LIGA / pa_rival
        for p in data[side]:
            eff = calc_eff(p)
            cred = max(5, min(15, round(0.8*eff*FD + 3)))
            out.append({
                "partido": pid,
                "equipo":  side,
                "nombre":  p["nombre"],
                "min":     p["min"],
                "pts":     p["puntos"],
                "eff":     round(eff*FD,1),
                "cred":    cred
            })
    time.sleep(DELAY)

# Guarda el JSON en Carnets/Documents
ruta = pathlib.Path.home() / "Documents" / "basket_data.json"
with open(ruta, "w", encoding="utf8") as f:
    json.dump(out, f, ensure_ascii=False, indent=2)

print(f"✅ JSON generado en {ruta} — {len(out)} registros")

✅ JSON generado en /private/var/mobile/Containers/Data/Application/E489EA38-32B7-4F3B-B70B-04343FABA908/Documents/basket_data.json — 0 registros
